# Life cycle assessment: hydrogen production and import routes

To start the Notebook please **run all cells ('Cell'-->'Run All')**. This will make the code disappear. Do not worry about the error below, as it will disapear, if you follow the instructions in the Notebook.

## 1. Introduction

Hydrogen (H<sub>2</sub>) is a central component of the future energy system. Burning hydrogen achieves a high temperature level, which is why hydrogen can replace coal and gas for high-temperature industrial processes. Moreover, it can be used as a basic component for creating synthetic fuels and other chemicals. In a fuel cell, hydrogen can be used galvanically to produce electricity for electric cars or in micro combined heat and power plants. There are several other application fields of hydrogen such as reverse power generation for peak load electricity or in district heating. 

Therefore, Germany and other industrialized countries have an interest in building up a hydrogen economy. For hydrogen production in Germany, the 2020 national hydrogen strategy (NWS) envisages an electrolysis capacity of 5 GW in 2030, corresponding to about 14 TWh or 424,000 tons of generated hydrogen - the 2021 coalition agreement has increased this goal to 10 GW. However, the NWS and most energy market studies from the literature forecast even a higher hydrogen demand of roughly 100 TWh (ca. 3 mil. tons) in 2030 and 500 TWh (ca. 15 mil. tons) in 2045 to meet the climate goals. Therefore, the import of hydrogen from other countries with higher production capacities can be essential for the future energy supply of Germany.

However, there are several possible hydrogen production and import routes. In order to meet the requirements of environmental sustainability, these routes must be examined comprehensively in terms of their environmental impact. This tool gives you the opportunity to assess and compare different pathways of H<sub>2</sub> interactively in terms of their global warming potential (GWP) which is calculated through a Life Cycle Assessment (LCA). In a LCA, all greenhouse gas emissions (in CO<sub>2</sub> equivalents) over the whole life cycle are considered. That includes emisions for the production of components as well as indirect emissions through electricity consumption or direct emissions in the processes. 

The H<sub>2</sub> routes are especially characterized by the hydrogen production, the transportation and the import country. Find out which pathway has the lowest GWP emissions and which pathway has an even higher footprint than continuing the use of fossil fuels. 

In [1]:
##Import of relevant libraries and modules
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label, IntSlider, Dropdown, RadioButtons, Layout, Style, interact, Button
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Javascript, clear_output
from termcolor import colored

In [2]:
#toggling code 
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')



In [3]:
def run_next(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    
def run_next_two(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+3)'))

def run_next_eleven(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+12)'))
     
def run_before(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()-3, IPython.notebook.get_selected_index())-1'))

def run_all_below(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
    
def run_current_and_all_below(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.ncells())'))
    
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(0, IPython.notebook.ncells())'))
    
def run_all_alternative():
    display(Javascript('IPython.notebook.execute_cell_range(0, IPython.notebook.ncells())'))




In [4]:
## general style definitions

CRED = '\033[91m'
CEND = '\033[0m'

button_style = {'button_color':'DodgerBlue', 'font_weight':'bold', }

In [5]:
file_name = 'Input_H2_Produktion_Import_Analyse.xlsx'
file_path = 'assets/excel'

##Importiere Daten mittels Pandas und setze default Werte fest

##Datenimport
#Entfernungen
M_Entfernung = pd.read_excel(file_path+'/'+file_name, sheet_name= 'Entfernungen', usecols='A:G', header= 0, nrows=9, index_col=0 )
M_E= pd.DataFrame(M_Entfernung)
#Strommixe
M_Strommix_Regionen = pd.read_excel(file_path+'/'+file_name, sheet_name= 'Strommix_Regionen', usecols='A:L', header= 0, nrows=20, index_col=0 )
M_Strom_GWP = pd.read_excel(file_path+'/'+file_name, sheet_name= 'GWP_Strom', usecols='A:L', header= 0, nrows=10, index_col=0 )
M_Strom_GWP = M_Strom_GWP.transpose()
#Auslastungen und Wirkungsgrad
M_Auslastung = pd.read_excel(file_path+'/'+file_name, sheet_name= 'Auslastung_Wirkungsgrad', usecols='A:J', header= 0, nrows=12, index_col=0 )
#GWP von Rohstoffen
M_Rohstoffe_GWP = pd.read_excel(file_path+'/'+file_name, sheet_name= 'GWP_Rohstoffe_Regionen', usecols='A:D', header= 0, nrows=10, index_col=0 )
#Charakterisierungsmatrix Q_GWP
Q_GWP = pd.read_excel(file_path+'/'+file_name, sheet_name= 'Charakterisierungsmatrix', usecols='A:B', header= 0, nrows=4, index_col=0 )
heizwert_h2 = 33.3 #[kWh/kg]

# 2. How many different H<sub>2</sub> routes do you want to analyze?

Choose a number of hydrogen production and/or import routes between 1 and 5 to analyze. Make sure to press the button after.

In [6]:
## Lege Anzahl der Vergleichsszenarien fest
anzahl_pfade_input = widgets.IntSlider(
    value = 3,
    min = 1,
    max = 5,
    orientation = 'horizontal',
    continuous_update = True,
)
anzahl_pfade_description_0 = 'How many hydrogen production and import routes do you want to analyze?'
anzahl_pfade_description_1 = 'Select a number between 1 and 5.'
anzahl_pfade_box_default = widgets.HBox([anzahl_pfade_input])
anzahl_pfade_box = widgets.VBox([anzahl_pfade_box_default])
auswahl_box = [widgets.VBox
    ([widgets.Label(value = anzahl_pfade_description_0), widgets.Label(value = anzahl_pfade_description_1), anzahl_pfade_box])
    ]
box = widgets.Accordion()
box.children = auswahl_box
box.set_title(0,'Number of analyzed hydrogen production and import routes')


button1 = widgets.Button(
    description='Set number of routes',
    layout=Layout(width='40%', height='40px'),
    button_style='primary',  #'primary', 'success', 'info', 'warning', 'danger', ''
    style= button_style,
    icon='',

    )

button1.on_click(run_next_two)


number_path_box = widgets.VBox([box,button1])

display(number_path_box)




<IPython.core.display.Javascript object>

# 3. Choose the parameters for your H<sub>2</sub> route?

You can choose the input data for all routes you want to analyze. The input data are divided in a basic data parameter input and the adjustment of electricity mixes.

**Basic data parameter:**

You can decide the **hydrogen production** (e.g. PEM electrolysis or stem methane reforming), the **production country**, the **transportation** (e.g. by pipeline or liquefied by ship) and the **electricity mixes** in the production country and Germany. If you choose PEM electrolysis for the hydrogen production, you can also change the efficiency and the full load hours. Please note, that you get a notification if the transportation method and the import country don't match. For instance, there won't be a pipeline between Australia and Germany. In those cases, you have to change your input data. If you choose Germany as the production country no transportation and conversion steps are required.

**Electricity mixes:**

You can adjust the electricity mixes in the **production country** and in **Germany**. These electricity mixes determine the GWP of the electricity used in the hydrogen production and import processes. Note, that the production process and the conversion always take place in the production country and the electricity mix of the production country is used for these processes. Especialy the PEM electrolysis needs a lot of electricity - in these cases the electricity mix of the production country might be the most relevant input parameter for the overall GWP calculation. The German electricity mix is only needed for the reconversion processes (e.g. regasification).


**Please make sure to click the blue button: 'Press to set basic input parameters'**

In [10]:

## Dropdowns for input values
# Festlegen der Anzahl der Routen, Alle Inputvariablen werden anzahl mal deklariert und sind daher in for schleifen. Beschreibene Variablen sind immer gleich und daher nicht in for schleifen
anzahl = anzahl_pfade_input.value

# Wasserstoffproduktion
h2_prod_input= {}
h2_prod ={}
for i in range (anzahl):
    h2_prod_input [i] = Dropdown(
        options=['PEM electrolysis', 'Steam methane reforming', 'Steam methane reforming + CCS', 'Coal gasification', 'Coal gasification + CCS']
    )
h2_prod_input_text = 'Choose the hydrogen production:'
h2_prod_input_titel = '𝗛𝘆𝗱𝗿𝗼𝗴𝗲𝗻 𝗽𝗿𝗼𝗱𝘂𝗰𝘁𝗶𝗼𝗻'

#Wasserstoffproduktion bei Elektrolyse
h2_prod_elektrolyse_text = 'If you choose PEM electrolysis, you can adjust specific input parameters:'
PEM_wirkungsgrad_input = {}
PEM_wirkungsgrad = {}
PEM_vollaststunden_input = {}
PEM_vollaststunden = {}
PEM_default_input = {}
PEM_default = {}
for i in range (anzahl):
    PEM_wirkungsgrad_input [i] = widgets.FloatSlider(
            value = 60,
            min = 1,
            max = 100,
            step = 0.1,
            orientation = 'horizontal',
            description = 'efficiency [%]',
            style ={'description_width': 'initial'}
        )
    PEM_vollaststunden_input [i] = widgets.FloatSlider(
            value = 4000,
            min = 10,
            max = 8760,
            step = 1,
            orientation = 'horizontal',
            description = 'full load hours [h/a]',
            style ={'description_width': 'initial'}
        )
    PEM_default_input [i] = widgets.HBox([PEM_wirkungsgrad_input [i], PEM_vollaststunden_input [i]])
    PEM_default [i] = widgets.HBox([PEM_default_input [i]])

# Produktionsland
prod_land_input = {}
prod_land = {}
for i in range (anzahl):
    prod_land_input [i]= Dropdown(
        options= ['Germany', 'Norway', 'United Kingdom', 'Spain', 'Algeria', 'Ghana', 'Qatar', 'Thailand', 'Australia']
)
prod_land_input_text = 'Choose the hydrogen production country:'
prod_land_input_titel = '𝗣𝗿𝗼𝗱𝘂𝗰𝘁𝗶𝗼𝗻 𝗰𝗼𝘂𝗻𝘁𝗿𝘆'

# Transportmittel
transport_input = {}
h2_transp = {}
for i in range (anzahl):
    transport_input [i] = Dropdown(
        options=['Ship (liquefied)', 'Ship (LOHC)', 'Pipeline', 'Road transport (gas)', 'Road transport (liquefied)', 'Road transport (LOHC)']
)
transport_input_text = 'Choose the hydrogen transportation method between the production country and Germany:'
transport_input_titel = '𝗧𝗿𝗮𝗻𝘀𝗽𝗼𝗿𝘁𝗮𝘁𝗶𝗼𝗻 𝗺𝗲𝘁𝗵𝗼𝗱'

# Strommix in Germany
strommix_de_input = {}
strommix_de = {}
for i in range (anzahl):
    strommix_de_input [i] = Dropdown(
        options=['Germany 2020', 'Germany 2030', 'Wind only', 'Solar only']
)
strommix_de_input_text = 'Choose the electricity mix for all electricity used in Germany: '
strommix_de_input_titel = '𝗘𝗹𝗲𝗰𝘁𝗿𝗶𝗰𝗶𝘁𝘆 𝗺𝗶𝘅 𝗶𝗻 𝗚𝗲𝗿𝗺𝗮𝗻𝘆'

# Detaillierter Strommix in Deutschland
#Kohle
strommix_de_input_kohle = {}
for i in range (anzahl):
    strommix_de_input_kohle[i] = widgets.FloatSlider(
    value = 0,
    #value = M_Strommix_Regionen.at[strommix_de, 'Coal'] * 100,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Coal',
    )
#Atom
strommix_de_input_atom = {}
for i in range (anzahl):
    strommix_de_input_atom[i] = widgets.FloatSlider(
    value = 0,
    # value = M_Strommix_Regionen.at[strommix_de, 'Nuclear'] * 100,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Nuclear',
    )
#Natural gas
strommix_de_input_gas = {}
for i in range (anzahl):
    strommix_de_input_gas[i] = widgets.FloatSlider(
    value = 0,
    # value = M_Strommix_Regionen.at[strommix_de, 'Natural gas'] * 100,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Natural gas',
    )
#Öl
strommix_de_input_oel = {}
for i in range (anzahl):
    strommix_de_input_oel[i] = widgets.FloatSlider(
    value = 0,
    # value = M_Strommix_Regionen.at[strommix_de, 'Oil'] * 100,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Oil',
    )
#Wind onshore
strommix_de_input_windon = {}
for i in range (anzahl):
    strommix_de_input_windon[i] = widgets.FloatSlider(
    # value = M_Strommix_Regionen.at[strommix_de, 'Wind onshore'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Wind on.',
    )
#Wind Offshore
strommix_de_input_windoff = {}
for i in range (anzahl):
    strommix_de_input_windoff[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_de, 'Wind offshore'] * 100,
    value =0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Wind off.',
    )
#Solar
strommix_de_input_solar = {}
for i in range (anzahl):
    strommix_de_input_solar[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_de, 'Solar'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Solar',
    )
#Wasserkraft
strommix_de_input_wasser = {}
for i in range (anzahl):
    strommix_de_input_wasser[i] = widgets.FloatSlider(
    value = 0,
    #value = M_Strommix_Regionen.at[strommix_de, 'Water'] * 100,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Water',
    )
#Biomasse
strommix_de_input_bio = {}
for i in range (anzahl):
    strommix_de_input_bio[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_de, 'Biomass'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Biomass',
    )
#Sonst. Erneuerbare
strommix_de_input_se = {}
for i in range (anzahl):
    strommix_de_input_se[i] = widgets.FloatSlider(
    value = 0,
    #value = M_Strommix_Regionen.at[strommix_de, 'Other renewables'] * 100,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Other ren.',
    )
#Sonstige
strommix_de_input_s = {}
for i in range (anzahl):
    strommix_de_input_s[i] = widgets.FloatSlider(
    value = 0,
    #value = M_Strommix_Regionen.at[strommix_de, 'Other'] * 100,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Other',
    )
#Ausgabe der Summe des Strommix (muss 100% ergeben)
strommix_de_input_summe = {}
for i in range (anzahl):
    strommix_de_input_summe[i] = widgets.FloatText(
        value = 100,
        step = .1,
        description = 'Sum [%]',
        disabled = True,
        continuous_update = True,
    )
#Predicted GWP
strommix_de_input_GWP_pred = {}
for i in range (anzahl):
    strommix_de_input_GWP_pred[i] = widgets.FloatText(
        value = 0,
        description = 'GWP [g CO2 eq./ kWh]',
        style ={'description_width': 'initial'},
        disabled = True,
        continuous_update = True,
    )

#Strommix Produktionsland
strommix_prod_input = {}
strommix_prod = {}
for i in range (anzahl):
    strommix_prod_input [i] = Dropdown(
        options=['Production country 2020', 'Production country 2030', 'Wind only', 'Solar only']
    )
strommix_prod_input_text = 'Choose the electricity mix for all electricity used in the hydrogen production country:'#(You can adjust the exact shares of different power plants in 
strommix_prod_input_titel = '𝗘𝗹𝗲𝗰𝘁𝗿𝗶𝗰𝗶𝘁𝘆 𝗺𝗶𝘅 𝗼𝗳 𝗽𝗿𝗼𝗱𝘂𝗰𝘁𝗶𝗼𝗻 𝗰𝗼𝘂𝗻𝘁𝗿𝘆'

#Detaillierter Strommix in Produktionsland
#Kohle
strommix_prod_input_kohle = {}
for i in range (anzahl):
    strommix_prod_input_kohle[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Coal'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Coal',
    )
#Atom
strommix_prod_input_atom = {}
for i in range (anzahl):
    strommix_prod_input_atom[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Nuclear'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Nuclear',
    )
#Natural gas
strommix_prod_input_gas = {}
for i in range (anzahl):
    strommix_prod_input_gas[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Natural gas'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Natural gas',
    )
#Öl
strommix_prod_input_oel = {}
for i in range (anzahl):
    strommix_prod_input_oel[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Oil'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Oil',
    )
#Wind onshore
strommix_prod_input_windon = {}
for i in range (anzahl):
    strommix_prod_input_windon[i] = widgets.FloatSlider(
    value = 0,
    #value = M_Strommix_Regionen.at[strommix_prod, 'Wind onshore'] * 100,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Wind on.',
    )
#Wind Offshore
strommix_prod_input_windoff = {}
for i in range (anzahl):
    strommix_prod_input_windoff[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Wind offshore'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Wind off.',
    )
#Solar
strommix_prod_input_solar = {}
for i in range (anzahl):
    strommix_prod_input_solar[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Solar'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Solar',
    )
#Wasserkraft
strommix_prod_input_wasser = {}
for i in range (anzahl):
    strommix_prod_input_wasser[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Water'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Water',
    )
#Biomasse
strommix_prod_input_bio = {}
for i in range (anzahl):
    strommix_prod_input_bio[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Biomass'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Biomass',
    )
#Sonst. Erneuerbare
strommix_prod_input_se = {}
for i in range (anzahl):
    strommix_prod_input_se[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Other renewables'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Other ren.',
    )
#Sonstige
strommix_prod_input_s = {}
for i in range (anzahl):
    strommix_prod_input_s[i] = widgets.FloatSlider(
    #value = M_Strommix_Regionen.at[strommix_prod, 'Other'] * 100,
    value = 0,
    min = 0,
    max = 100,
    step = 0.1,
    continuous_update = False,
    orientation = 'vertical',
    description = 'Other',
    )
#Ausgabe der Summe des Strommix (muss 100% ergeben)
strommix_prod_input_summe = {}
for i in range (anzahl):
    strommix_prod_input_summe[i] = widgets.FloatText(
        value = 100,
        description = 'Sum [%]',
        disabled = True,
        continuous_update = True,
    )
#Predicted GWP
strommix_prod_input_GWP_pred = {}
for i in range (anzahl):
    strommix_prod_input_GWP_pred[i] = widgets.FloatText(
        value = 0,
        description = 'GWP [g CO2 eq./ kWh]',
        style ={'description_width': 'initial'},
        disabled = True,
        continuous_update = True,
    )

##Widgets zur Anzeige der eingegebenen Parameter
#H2 prod
def_h2_prod = {}
for i in range (anzahl):
    def_h2_prod[i] = widgets.Text(
        value= 'Press button above',
        description = 'H2 production:',
        style ={'description_width': 'initial'},
        layout = Layout(width = '400px'),
        disabled = True
    )
#Production country
def_prod_land = {}
for i in range (anzahl):
    def_prod_land[i] = widgets.Text(
        value= 'Press button above',
        style ={'description_width': 'initial'},
        layout = Layout(width = '400px'),
        description = '   Production country:',
        disabled = True
    )
#Transport
def_h2_transp = {}
for i in range (anzahl):
    def_h2_transp[i] = widgets.Text(
        value= 'Press button above',
        description = '   Transportation method:',
        style ={'description_width': 'initial'},
        layout = Layout(width = '400px'),
        disabled = True
    )
#Strommix Prod
def_strom_prod = {}
for i in range (anzahl):
    def_strom_prod[i] = widgets.Text(
        value= 'Press button above',
        description = 'Basic input:',
        style ={'description_width': 'initial'},
        layout = Layout(width = '300px'),
        disabled = True
    )
#Strommix Germany
def_strom_de = {}
for i in range (anzahl):
    def_strom_de[i] = widgets.Text(
        value= 'Press button above',
        description = 'Basic input:',
        style ={'description_width': 'initial'},
        layout = Layout(width = '300px'),
        disabled = True
    )


#Button zur Aktualisierung von basic input
button_basic_input = {}
for i in range (anzahl):
    button_basic_input [i] = widgets.Button(
        description = 'Press to set basic input parameters',
 
        layout=Layout(width='40%', height='40px'),
        button_style='primary',  #'primary', 'success', 'info', 'warning', 'danger', ''
        style= button_style,
    )

#Eventhandler von Basic Input
#Variablen deklarieren
ungueltig ={}                   #Variable für ungülzige Transportmethode
entfernung ={}                  #Variable für die Transportentfernung
V_Strommix_de = {}              #Variable für Strommix in De
strom_GWP_de = {}               #Variable für strom GWP in De
V_Strommix_prod = {}            #Variable für Strommix in Prodland
strom_GWP_prod = {}             #Variable für strom GWP in Prodland

out= widgets.Output()

def on_button_basic_input_clicked(sender): ### festsetz Button! TO DO: Nach unten ziehen
    #clear_output()
    
    for i in range (anzahl):
        #Einlesen der Daten
        prod_land[i] = prod_land_input[i].value
        h2_prod[i] = h2_prod_input[i].value
        h2_transp[i] = transport_input[i].value
        strommix_de[i] = strommix_de_input[i].value
        strommix_prod[i] = strommix_prod_input[i].value
        
        #PEM electrolysis
        if h2_prod[i] == 'PEM electrolysis':
            PEM_vollaststunden[i] = PEM_vollaststunden_input[i].value
            PEM_wirkungsgrad[i] = PEM_wirkungsgrad_input[i].value/100

        ## Festlegen des Strommix im Produktionsland
        if strommix_prod[i] == 'Production country 2020':
            strommix_prod[i] = prod_land[i] + ' 2020'
        if strommix_prod[i] == 'Production country 2030':
            strommix_prod[i] = prod_land[i] + ' 2030'
 
        ##Überprüfen, ob Transport für Importland geeignet
        ungueltig[i] = M_E.isnull()
        ungueltig[i]= ungueltig[i].at[prod_land[i], h2_transp[i]]
        if ungueltig[i] == True :
            print(CRED + 'Route '+str(i+1)+': Transportation method not possible for selected country! Please choose an appropiate transportation method'+ CEND)
        else:
            print('Route '+str(i+1)+': Selected transportation method possible')

        ##Festsetzen der Transportlängen
        entfernung [i] = M_Entfernung [h2_transp[i]] [prod_land[i]]
        ##Festsetzen der Default Strommixe aus Importdaten
        V_Strommix_de[i] = M_Strommix_Regionen.loc[strommix_de[i]]	#Germany
        strom_GWP_de [i] = V_Strommix_de [i] @ M_Strom_GWP ['Germany']
        V_Strommix_prod [i] = M_Strommix_Regionen.loc[strommix_prod [i]]	#Produktionsland
        strom_GWP_prod [i] = V_Strommix_prod [i] @ M_Strom_GWP [prod_land[i]]
        
        ##Aktualisiere Strommix in Deutschland
        strommix_de_input_kohle[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Coal'] * 100
        strommix_de_input_atom[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Nuclear'] * 100
        strommix_de_input_gas[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Natural gas'] * 100
        strommix_de_input_oel[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Oil'] * 100
        strommix_de_input_windon[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Wind onshore'] * 100
        strommix_de_input_windoff[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Wind offshore'] * 100
        strommix_de_input_solar[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Solar'] * 100
        strommix_de_input_wasser[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Water'] * 100
        strommix_de_input_bio[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Biomass'] * 100
        strommix_de_input_se[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Other renewables'] * 100
        strommix_de_input_s[i].value = M_Strommix_Regionen.at[strommix_de[i], 'Other'] * 100

        ##Aktualisiere Strommix in Produktionsland
        strommix_prod_input_kohle[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Coal'] * 100
        strommix_prod_input_atom[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Nuclear'] * 100
        strommix_prod_input_gas[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Natural gas'] * 100
        strommix_prod_input_oel[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Oil'] * 100
        strommix_prod_input_windon[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Wind onshore'] * 100
        strommix_prod_input_windoff[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Wind offshore'] * 100
        strommix_prod_input_solar[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Solar'] * 100
        strommix_prod_input_wasser[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Water'] * 100
        strommix_prod_input_bio[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Biomass'] * 100
        strommix_prod_input_se[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Other renewables'] * 100
        strommix_prod_input_s[i].value = M_Strommix_Regionen.at[strommix_prod[i], 'Other'] * 100
    
        #Default Werte
        def_h2_prod[i].value = h2_prod[i]
        def_h2_transp[i].value = h2_transp[i]
        def_prod_land[i].value = prod_land[i]
        if strommix_de[i] == 'Solar only':
            def_strom_de[i].value = 'Solar only (Germany)'
        elif strommix_de[i] == 'Wind only':
            def_strom_de[i].value = 'Wind only (Germany)'
        else:
            def_strom_de[i].value = strommix_de[i]
        if strommix_prod[i] == 'Solar only':
            def_strom_prod[i].value = 'Solar only (' + prod_land[i]+ ')'
        elif strommix_prod[i] == 'Wind only':
            def_strom_prod[i].value = 'Wind only (' + prod_land[i]+ ')'
        else:
            def_strom_prod[i].value = strommix_prod[i]

for i in range (anzahl):
    button_basic_input[i].on_click(on_button_basic_input_clicked)

#Button zur Berechnung der Summe (Summe muss 100% ergeben) und des GWP im Produktionsland
button_prod_summe = {}
for i in range (anzahl):
    button_prod_summe [i] = widgets.Button(
        description = 'Calculate sum and GWP',
        layout = Layout(width = '200px'),
        color = 'lightgreen'
    )

#Eventhandler des Buttons zur Berechnung der Summe und des GWP im Produktionsland
def on_button_prod_summe_clicked(sender):
    for i in range (anzahl):
        # Werte übernehmen
        strommix_de[i] = strommix_de_input[i].value
        strommix_prod[i] = strommix_prod_input[i].value
        prod_land[i] = prod_land_input[i].value
        ## Festlegen des Strommix im Produktionsland
        if strommix_prod[i] == 'Production country 2020':
            strommix_prod[i] = prod_land[i] + ' 2020'
        if strommix_prod[i] == 'Production country 2030':
            strommix_prod[i] = prod_land[i] + ' 2030'
        #Summe berechnen
        strommix_prod_input_summe[i].value = round((strommix_prod_input_kohle[i].value + strommix_prod_input_atom[i].value + strommix_prod_input_gas[i].value + strommix_prod_input_oel[i].value + strommix_prod_input_windon[i].value + strommix_prod_input_windoff[i].value +strommix_prod_input_solar[i].value + strommix_prod_input_wasser[i].value + strommix_prod_input_bio[i].value + strommix_prod_input_se[i].value + strommix_prod_input_s[i].value),0)
        #GWP berechnen
        V_Strommix_prod[i] = M_Strommix_Regionen.loc[strommix_prod[i]]	#Produktionsland
        strom_GWP_prod[i] = V_Strommix_prod[i] @ M_Strom_GWP [prod_land[i]]
        V_Strommix_prod[i].at['Coal'] = strommix_prod_input_kohle[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Nuclear'] = strommix_prod_input_atom[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Natural gas'] = strommix_prod_input_gas[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Oil'] = strommix_prod_input_oel[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Wind onshore'] = strommix_prod_input_windon[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Wind offshore'] = strommix_prod_input_windoff[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Solar'] = strommix_prod_input_solar[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Water'] = strommix_prod_input_wasser[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Biomass'] = strommix_prod_input_bio[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Other renewables'] = strommix_prod_input_se[i].value/strommix_prod_input_summe[i].value
        V_Strommix_prod[i].at['Other'] = strommix_prod_input_s[i].value/strommix_prod_input_summe[i].value
        strommix_prod_input_GWP_pred[i].value = round((V_Strommix_prod[i] @ M_Strom_GWP [prod_land[i]] * 1000),0)
for i in range (anzahl):
    button_prod_summe[i].on_click(on_button_prod_summe_clicked)

#Button zur Berechnung der Summe (Summe muss 100% ergeben) und des GWP in Deutschland
button_de_summe = {}
for i in range (anzahl):
    button_de_summe[i] = widgets.Button(
        description = 'Calculate sum and GWP',
        layout = Layout(width = '200px'),
        color = 'lightgreen'
    )

#Eventhandler des Buttons zur Berechnung der Summe und des GWP in Deutschland
def on_button_de_summe_clicked(sender):
    for i in range (anzahl):
        strommix_de[i] = strommix_de_input[i].value
        strommix_prod[i] = strommix_prod_input[i].value
        prod_land[i] = prod_land_input[i].value
        ## Festlegen des Strommix im Produktionsland
        if strommix_prod[i] == 'Production country 2020':
            strommix_prod[i] = prod_land[i] + ' 2020'
        if strommix_prod[i] == 'Production country 2030':
            strommix_prod[i] = prod_land[i] + ' 2030'
        #Summe berechnen
        strommix_de_input_summe[i].value = round((strommix_de_input_kohle[i].value + strommix_de_input_atom[i].value + strommix_de_input_gas[i].value + strommix_de_input_oel[i].value + strommix_de_input_windon[i].value + strommix_de_input_windoff[i].value +strommix_de_input_solar[i].value + strommix_de_input_wasser[i].value + strommix_de_input_bio[i].value + strommix_de_input_se[i].value + strommix_de_input_s[i].value),0)
        #GWP berechnen
        V_Strommix_de[i] = M_Strommix_Regionen.loc[strommix_de[i]]	#Germany
        strom_GWP_de [i]= V_Strommix_de[i] @ M_Strom_GWP ['Germany']
        V_Strommix_de[i].at['Coal'] = strommix_de_input_kohle[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Nuclear'] = strommix_de_input_atom[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Natural gas'] = strommix_de_input_gas[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Oil'] = strommix_de_input_oel[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Wind onshore'] = strommix_de_input_windon[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Wind offshore'] = strommix_de_input_windoff[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Solar'] = strommix_de_input_solar[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Water'] = strommix_de_input_wasser[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Biomass'] = strommix_de_input_bio[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Other renewables'] = strommix_de_input_se[i].value/strommix_de_input_summe[i].value
        V_Strommix_de[i].at['Other'] = strommix_de_input_s[i].value/strommix_de_input_summe[i].value
        strommix_de_input_GWP_pred[i].value = round((V_Strommix_de[i]@ M_Strom_GWP['Germany'] * 1000),0)
for i in range (anzahl):
    button_de_summe[i].on_click(on_button_de_summe_clicked)

##Erstellen der Textboxen


## Basic Input parameter Box
basic_input_parameter_box = {}
for i in range (anzahl):
    basic_input_parameter_box[i] = widgets.VBox([
            widgets.Label(value = h2_prod_input_titel), widgets.VBox([Label(h2_prod_input_text), h2_prod_input[i]]),
            widgets.Label(value = h2_prod_elektrolyse_text), widgets.VBox([PEM_default[i]]),
            widgets.Label(value = prod_land_input_titel), widgets.VBox([Label(prod_land_input_text), prod_land_input[i]]),
            widgets.Label(value = strommix_prod_input_titel), widgets.VBox([Label(strommix_prod_input_text), strommix_prod_input[i]]),
            widgets.Label(value = transport_input_titel), widgets.VBox([Label(transport_input_text), transport_input[i]]),
            widgets.Label(value = strommix_de_input_titel), widgets.VBox([Label(strommix_de_input_text), strommix_de_input[i]]),
            
        
        ])



#Ein Tab für jede Route
basic_input_parameter_box_disp = [basic_input_parameter_box [key] for key in basic_input_parameter_box]
basic_input_parameter_box_tab = widgets.Tab()
basic_input_parameter_box_tab.children = basic_input_parameter_box_disp
for i in range(anzahl):
    basic_input_parameter_box_tab.set_title(i, 'Route ' + str(i+1))
    
basic_input_parameter_box_button = button_basic_input[0] 
basic_input_parameter_box_final = widgets.VBox([basic_input_parameter_box_tab, basic_input_parameter_box_button])

## Strommix adjustment box

#Titel und Beschreibung
strommix_adjustment_description_0 = 'Adjust the electricity mix here:'
strommix_adjustment_description_1 = 'Please ensure that the sum of the electricity generation mix is 100%. (Test with buttons below)'
strommix_prod_default_titel = '𝗘𝗹𝗲𝗰𝘁𝗿𝗶𝗰𝗶𝘁𝘆 𝗺𝗶𝘅 𝗽𝗿𝗼𝗱𝘂𝗰𝘁𝗶𝗼𝗻 𝗰𝗼𝘂𝗻𝘁𝗿𝘆 [%]'
strommix_de_default_titel = '𝗘𝗹𝗲𝗰𝘁𝗿𝗶𝗰𝗶𝘁𝘆 𝗺𝗶𝘅 𝗚𝗲𝗿𝗺𝗮𝗻𝘆 [%]'

#Aufbau der Box
strommix_prod_default_input = {}
strommix_prod_default = {}
strommix_de_default_input = {}
strommix_de_default = {}
strommix_adjustment_box = {}

for i in range (anzahl):
    strommix_prod_default_input[i] = widgets.HBox([strommix_prod_input_kohle[i], strommix_prod_input_atom[i], strommix_prod_input_gas[i], strommix_prod_input_oel[i], strommix_prod_input_windon[i], strommix_prod_input_windoff[i], strommix_prod_input_solar[i], strommix_prod_input_wasser[i], strommix_prod_input_bio[i], strommix_prod_input_se[i], strommix_prod_input_s[i]])
    strommix_prod_default[i] = widgets.VBox([widgets.Label(value = strommix_prod_default_titel), def_strom_prod[i], strommix_prod_default_input[i], widgets.HBox([button_prod_summe[i], strommix_prod_input_summe[i], strommix_prod_input_GWP_pred[i]])])
    strommix_de_default_input[i] = widgets.HBox([strommix_de_input_kohle[i], strommix_de_input_atom[i], strommix_de_input_gas[i], strommix_de_input_oel[i], strommix_de_input_windon[i], strommix_de_input_windoff[i], strommix_de_input_solar[i], strommix_de_input_wasser[i], strommix_de_input_bio[i], strommix_de_input_se[i], strommix_de_input_s[i]])
    strommix_de_default[i] = widgets.VBox([widgets.Label(value = strommix_de_default_titel), def_strom_de[i], strommix_de_default_input[i], widgets.HBox([button_de_summe[i], strommix_de_input_summe[i], strommix_de_input_GWP_pred[i]])])
    strommix_adjustment_box[i] = widgets.VBox([
            widgets.Label(value = 'You selected:'),
            widgets.HBox([def_h2_prod[i], def_prod_land[i], def_h2_transp[i]]),
            widgets.Label(value = strommix_adjustment_description_0), widgets.Label(value = strommix_adjustment_description_1),
            strommix_prod_default[i],
            strommix_de_default[i]
        ])

#Ein Tab für jede Route
strommix_adjustment_box_disp = [strommix_adjustment_box [key] for key in strommix_adjustment_box]
strommix_adjustment_box_tab = widgets.Tab()
strommix_adjustment_box_tab.children = strommix_adjustment_box_disp
for i in range(anzahl):
    strommix_adjustment_box_tab.set_title(i, 'Route ' + str(i+1))

#Gesamtbox
gesamtbox_children = [widgets.VBox([basic_input_parameter_box_final]), widgets.VBox([strommix_adjustment_box_tab])]
gesamtbox_acc = widgets.Accordion()
gesamtbox_acc.children = gesamtbox_children
gesamtbox_acc.set_title(0, 'Basic input parameters')
gesamtbox_acc.set_title(1, 'Adjustment of electricity mixes')

#display(basic_input_parameter_box_tab)
display(gesamtbox_acc)

Accordion(children=(VBox(children=(VBox(children=(Tab(children=(VBox(children=(Label(value='𝗛𝘆𝗱𝗿𝗼𝗴𝗲𝗻 𝗽𝗿𝗼𝗱𝘂𝗰𝘁𝗶𝗼…

Route 1: Selected transportation method possible
Route 2: Selected transportation method possible
Route 3: Selected transportation method possible
Route 4: Selected transportation method possible
Route 1: Selected transportation method possible
Route 2: Selected transportation method possible
Route 3: Selected transportation method possible
Route 4: Transportation method not possible for selected country! Please choose an appropiate transportation method
Route 1: Selected transportation method possible
Route 2: Selected transportation method possible
Route 3: Selected transportation method possible
Route 4: Selected transportation method possible


# 4. Calculate the GWP of your chosen hydrogen routes

Depending on the routes you choose the in- and output flows for the stages **hydrogen production**, **conversion**, **transportation** as well as **losses and utilization** are quantified and the GWP is calculated based on the impact assessment method ReCiPE 2016 (H).

In [8]:
button2 = widgets.Button(
    description='Calculate GWP of routes',
    layout=Layout(width='40%', height='40px'),
    button_style='primary',  #'primary', 'success', 'info', 'warning', 'danger', ''
    style= button_style,
    icon='',

    )

button2.on_click(run_all_below)
display(button2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(button_style='primary', description='Calculate GWP of routes', layout=Layout(height='40px', width='40%'…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
### GWP of H2 production

## Übernehmen der Inputdaten

#Standarddaten
for i in range (anzahl):
        h2_prod[i] = h2_prod_input[i].value
        prod_land[i] = prod_land_input[i].value
        h2_transp[i] = transport_input[i].value
        strommix_de[i] = strommix_de_input[i].value
        strommix_prod[i] = strommix_prod_input[i].value
        if strommix_prod[i] == 'Production country 2020':
                strommix_prod[i] = prod_land[i] + ' 2020'
        if strommix_prod[i] == 'Production country 2030':
            strommix_prod[i] = prod_land[i] + ' 2030'
        if strommix_prod[i] == 'Germany':
                strommix_prod[i] = 'Germany 2020'
        PEM_vollaststunden[i] = PEM_vollaststunden_input[i].value
        PEM_wirkungsgrad[i] = PEM_wirkungsgrad_input[i].value/100
        entfernung[i] = M_Entfernung [h2_transp[i]] [prod_land[i]]

#GWP Strom Produktionsland
strommix_prod_summe = {}
for i in range (anzahl):
        strommix_prod_summe[i] = strommix_prod_input_kohle[i].value + strommix_prod_input_atom[i].value + strommix_prod_input_gas[i].value + strommix_prod_input_oel[i].value + strommix_prod_input_windon[i].value + strommix_prod_input_windoff[i].value +strommix_prod_input_solar[i].value + strommix_prod_input_wasser[i].value + strommix_prod_input_bio[i].value + strommix_prod_input_se[i].value + strommix_prod_input_s[i].value
        #GWP berechnen
        V_Strommix_prod[i] = M_Strommix_Regionen.loc[strommix_prod[i]]	#Produktionsland
        strom_GWP_prod[i] = V_Strommix_prod[i] @ M_Strom_GWP [prod_land [i]]
        V_Strommix_prod[i].at['Coal'] = strommix_prod_input_kohle[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Nuclear'] = strommix_prod_input_atom[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Natural gas'] = strommix_prod_input_gas[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Oil'] = strommix_prod_input_oel[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Wind onshore'] = strommix_prod_input_windon[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Wind offshore'] = strommix_prod_input_windoff[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Solar'] = strommix_prod_input_solar[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Water'] = strommix_prod_input_wasser[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Biomass'] = strommix_prod_input_bio[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Other renewables'] = strommix_prod_input_se[i].value/strommix_prod_summe[i]
        V_Strommix_prod[i].at['Other'] = strommix_prod_input_s[i].value/strommix_prod_summe[i]
        strom_GWP_prod[i] = V_Strommix_prod[i] @ M_Strom_GWP [prod_land [i]]

#GWP Strom Deutschland
strommix_de_summe = {}
for i in range (anzahl):
        strommix_de_summe[i] = strommix_de_input_kohle[i].value + strommix_de_input_atom[i].value + strommix_de_input_gas[i].value + strommix_de_input_oel[i].value + strommix_de_input_windon[i].value + strommix_de_input_windoff[i].value +strommix_de_input_solar[i].value + strommix_de_input_wasser[i].value + strommix_de_input_bio[i].value + strommix_de_input_se[i].value + strommix_de_input_s[i].value
        #GWP berechnen
        V_Strommix_de[i] = M_Strommix_Regionen.loc[strommix_de[i]]	#Germany
        strom_GWP_de[i] = V_Strommix_de[i] @ M_Strom_GWP ['Germany']
        V_Strommix_de[i].at['Coal'] = strommix_de_input_kohle[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Nuclear'] = strommix_de_input_atom[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Natural gas'] = strommix_de_input_gas[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Oil'] = strommix_de_input_oel[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Wind onshore'] = strommix_de_input_windon[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Wind offshore'] = strommix_de_input_windoff[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Solar'] = strommix_de_input_solar[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Water'] = strommix_de_input_wasser[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Biomass'] = strommix_de_input_bio[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Other renewables'] = strommix_de_input_se[i].value/strommix_de_summe[i]
        V_Strommix_de[i].at['Other'] = strommix_de_input_s[i].value/strommix_de_summe[i]
        strom_GWP_de[i] = V_Strommix_de[i] @ M_Strom_GWP ['Germany']

### GWP H2 Produktion

##Erstellen und Anpassen der Interventionsmatrix B_h2_prod

B_h2_prod = {}
PEM_stack_durchsatz_lifetime = {}
PEM_plant_durchsatz_lifetime = {}
for i in range (anzahl):
        B_h2_prod [i] = pd.read_excel(file_path+'/'+file_name, sheet_name= 'H2 Produktion_Input', usecols='A,C:J', header= 0, nrows=13, index_col=0 )
        #B_h2_prod [i] = M_B_h2_prod
        #Passe Wirkungsgrad von PEM-Elektrolyse an
        B_h2_prod[i].at['Electricity (vor Wirkungsgrad)', 'PEM electrolysis'] = B_h2_prod[i].at['Electricity (vor Wirkungsgrad)', 'PEM electrolysis'] / PEM_wirkungsgrad[i]
        #Passe Materialfaktor von items über Vollaststunden und H2 Durchsatz Lifetime an
        #PEM Stack 1MW
        PEM_stack_durchsatz_lifetime[i] = PEM_vollaststunden[i]* PEM_wirkungsgrad[i]* M_Auslastung.at['PEM Stack 1MW', 'Lebensdauer']* M_Auslastung.at['PEM Stack 1MW', 'Leistung [MW]'] *1000/heizwert_h2
        B_h2_prod[i].at['PEM Stack 1MW', 'PEM electrolysis' ] = B_h2_prod[i].at['PEM Stack 1MW', 'PEM electrolysis' ] / PEM_stack_durchsatz_lifetime[i]
        #PEM Plant 1 MW
        PEM_plant_durchsatz_lifetime[i] = PEM_vollaststunden[i]* PEM_wirkungsgrad[i]* M_Auslastung.at['PEM Plant 1MW', 'Lebensdauer']* M_Auslastung.at['PEM Plant 1MW', 'Leistung [MW]'] *1000/heizwert_h2
        B_h2_prod[i].at['PEM Plant 1MW', 'PEM electrolysis' ] = B_h2_prod[i].at['PEM Plant 1MW', 'PEM electrolysis' ] / PEM_plant_durchsatz_lifetime[i]
        #SMR plant
        B_h2_prod[i].at['SMR Plant', 'Steam methane reforming'] =B_h2_prod[i].at['SMR Plant', 'Steam methane reforming'] / M_Auslastung.at['SMR Plant', 'H2 Durchsatz Lifetime [kg]']
        B_h2_prod[i].at['SMR Plant', 'Steam methane reforming + CCS'] =B_h2_prod[i].at['SMR Plant', 'Steam methane reforming + CCS'] / M_Auslastung.at['SMR Plant', 'H2 Durchsatz Lifetime [kg]']
        ##Daten für CCS plant bisher nicht berücksichtigt
        #CGR plant
        B_h2_prod[i].at['CGR Plant', 'Coal gasification'] =B_h2_prod[i].at['CGR Plant', 'Coal gasification'] / M_Auslastung.at['CGR Plant', 'H2 Durchsatz Lifetime [kg]']
        B_h2_prod[i].at['CGR Plant', 'Coal gasification + CCS'] =B_h2_prod[i].at['CGR Plant', 'Coal gasification + CCS'] / M_Auslastung.at['CGR Plant', 'H2 Durchsatz Lifetime [kg]']
        #Auswahl der H2 Produktion
        B_h2_prod[i] = B_h2_prod[i][h2_prod[i]]
        #Transponiere B
        B_h2_prod[i] = B_h2_prod[i].transpose()

##Erstellen und Anpassungen der Charakterisierungsmatrix Q_h2_prod_GWP
Q_h2_prod_GWP = {}
for i in range (anzahl):
        Q_h2_prod_GWP[i] = pd.read_excel(file_path+'/'+file_name, sheet_name= 'H2 Produktion_Input_Wirkung', usecols='A,C', header= 0, nrows=13, index_col=0 )
        #Q_h2_prod_GWP[i] = M_Q_h2_prod_GWP
        Q_h2_prod_GWP[i].at['Water', 'GWP'] = M_Rohstoffe_GWP.at[prod_land[i], 'Water'] #Passe GWP von Wasser an
        Q_h2_prod_GWP[i].at['Electricity (vor Wirkungsgrad)', 'GWP'] = strom_GWP_prod[i]    #Passe GWP Strommix an
        Q_h2_prod_GWP[i].at['Electricity', 'GWP'] = strom_GWP_prod [i]                     #Passe GWP Strommix an
        Q_h2_prod_GWP[i].at['Natural gas', 'GWP'] = M_Rohstoffe_GWP.at[prod_land[i], 'Natural gas'] #Passe GWP Methan an
        Q_h2_prod_GWP[i].at['Coal', 'GWP'] = M_Rohstoffe_GWP.at[prod_land[i], 'Coal']   #Passe GWP Kohle an


##Bestimmung des indirekten GWP der H2 Produktion über Multiplikation von Q_h2_prod_GWP,B_h2_prod
h2_prod_GWP_ind = {}
for i in range (anzahl):
        h2_prod_GWP_ind [i]=B_h2_prod[i] @ Q_h2_prod_GWP [i]      #Skallierung durch Verluste und Nutzung wird in späteren Schritt berücksichtigt

##Bestimmung des direkten GWP der H2 Produktion durch Outputmatrix O_h2_prod, Charakterisierungsmatrix Q_GWP
O_h2_prod = {}
h2_prod_GWP_dir = {}
for i in range (anzahl):
        O_h2_prod[i] = pd.read_excel(file_path+'/'+file_name, sheet_name= 'H2 Produktion_Output', usecols='A,C:J', header= 0, nrows=4, index_col=0 )
        #O_h2_prod[i] = M_O_h2_prod
        #Auswahl der H2 Produktion
        O_h2_prod[i]=O_h2_prod[i][h2_prod[i]]
        #Transponiere Outputmatrix
        O_h2_prod [i]=O_h2_prod[i].transpose()
        h2_prod_GWP_dir[i]=O_h2_prod [i] @ Q_GWP       #Skallierung durch Verluste und Nutzung wird in späteren Schritt berücksichtigt

#Ermittlung von GWP gesamt
h2_prod_GWP = {}
for i in range (anzahl):
        h2_prod_GWP[i]=h2_prod_GWP_dir[i]+h2_prod_GWP_ind[i]
        # print('The GWP of the hydrogen production is ', h2_prod_GWP[i] , ' kg CO2/ kg H2')
        # print('The indirect GWP of the hydrogen production is ', h2_prod_GWP_ind[i] , ' kg CO2/ kg H2')
        # print('The direct GWP of the hydrogen production is ', h2_prod_GWP_dir[i] , ' kg CO2/ kg H2')


In [17]:
### GWP of H2 Conversions

##Festlegen der Umwandlungsschritte basierend auf gewählten Transport
h2_umw = {}
for i in range (anzahl):
    if h2_transp[i] == 'Ship (liquefied)' or h2_prod_input[i] == 'Road transport (liquefied)' :
        h2_umw[i] = ['Liquefaction', 'Regasification']
    elif h2_transp[i] == 'Pipeline' or h2_prod_input[i] == 'Road transport (gas)' :
        h2_umw[i] = ['Compression', 'Relaxation']
    elif h2_transp[i] == 'Ship (LOHC)' or h2_prod_input[i] == 'Road transport (LOHC)' :
        h2_umw[i] = ['Hydrogenation (H2 to LOHC)', 'Dehydrogenation (LOHC to H2)']

##Import der Interventionsmatrix B_h2_umw
B_h2_umw = {}
for i in range (anzahl):
    B_h2_umw[i] = pd.read_excel(file_path+'/'+file_name, sheet_name= 'Umwandlungen_Input', usecols='A,C:H', header= 0, nrows=13, index_col=0 )
    #Auswahl der Umwandlung
    B_h2_umw[i] = B_h2_umw[i][h2_umw[i]]
    #Transponiere B_h2_umw
    B_h2_umw[i] = B_h2_umw[i].transpose()

##Anpassungen der Charakterisierungsmatrix Q_h2_umw_GWP
Q_h2_umw_GWP = {}
for i in range (anzahl):
    Q_h2_umw_GWP[i] = pd.read_excel(file_path+'/'+file_name, sheet_name= 'Umwandlungen_Input_Wirkung', usecols='A,C', header= 0, nrows=13, index_col=0 )
    'Anpassung Strom'
    Q_h2_umw_GWP[i].at['Electricity production country', 'GWP'] = Q_h2_umw_GWP[i].at['Electricity production country', 'GWP'] * strom_GWP_prod[i]
    Q_h2_umw_GWP[i].at['Electricity Germany', 'GWP'] = Q_h2_umw_GWP[i].at['Electricity Germany', 'GWP'] * strom_GWP_de[i]

##Bestimmung der indirekten GWP Matrix der H2 Umwandlung über Multiplikation von B_h2_umw, Q_h2_umw_GWP und Skalierungsfaktor
h2_umw_GWP_ind = {}
for i in range (anzahl):
    h2_umw_GWP_ind[i] = B_h2_umw[i] @ Q_h2_umw_GWP[i]            #Es fehlt noch Skalierungsfaktor
    #print(h2_umw_GWP_ind[i])



In [18]:
### GWP of H2 Transportation

M_transp = {}
O_h2_transp_GWP = {}
h2_transp_GWP_dir = {}
for i in range (anzahl):
    #Import des direkten GWP durch Transport
    M_transp[i] = pd.read_excel(file_path+'/'+file_name, sheet_name= 'Transport_Output_Wirkung', usecols='A,C:H', header= 0, nrows=2, index_col=0 )
    O_h2_transp_GWP[i] = M_transp[i].loc[['GWP']]
    ## Auswahl des Transports und Ermittlung des GWP mittels O_h2_transp_GWP, entfernung und Skalierungsfaktor
    O_h2_transp_GWP[i] = O_h2_transp_GWP[i][h2_transp[i]]
    O_h2_transp_GWP[i] = O_h2_transp_GWP[i].transpose()
    h2_transp_GWP_dir[i] = O_h2_transp_GWP[i] * entfernung[i]
    #print(h2_transp_GWP_dir[i])

In [19]:
### H2 losses and utilization

### Ermittle Skalierungsfaktoren für Wasserstoffverlust in Umwandlungschritten und Gebrauch für Wärmegenerierung

#Deklaration der Variablen für Verluste Nutzung [alle in Prozent], Beschreibung im Code bei der Berechnung
r_umw = {}
verl_r_umw = {}
nutz_r_umw  = {}
verl_transp_rel = {}
verl_transp = {}
nutz_transp = {}
v_umw = {}
verl_v_umw = {}
nutz_v_umw = {}
#Deklaration der Variablen für das skallierte GWP in den Produktionsstufen, welches entandenes GWP in den Vorstufen für Nutzung und Verluste in der Stufe miteinbezieht
h2_prod_GWP_s = {}       #Produktion   
h2_v_umw_GWP_s = {}        #Vorumwandlung
h2_transp_GWP_s = {}        #Transport
h2_r_umw_GWP_s = {}         #Rückumwandlung

for i in range (anzahl):
    #Rückumwandlung
    # from re import X
    # from xml.etree.ElementPath import xpath_tokenizer
    r_umw [i] = h2_umw [i] [1]
    verl_r_umw[i] = B_h2_umw[i] ['Hydrogen loss'] [r_umw[i]]                             #Verluste
    nutz_r_umw[i] = B_h2_umw[i] ['Hydrogen use for heat gen'] [r_umw[i]]                 #Nutzung für Wärme

    #Transport
    verl_transp_rel[i] = M_transp[i] [h2_transp[i]] ['Hydrogen loss']                    #Verluste in % pro 100km
    verl_transp[i] = ((1+verl_transp_rel[i]/100) ** (entfernung[i]*10) - 1) * 100        #Verluste in 5 auf der gesamten Strecke, Entfernung ist in 1000 km gegeben
    nutz_transp[i] = 0                                                             #Nutzung für Antrieb kann noch implementiert werden, ggf. muss dafür Rückumwandlung berücksichtigt werden

    #(Vor-)Umwandlung
    v_umw[i] = h2_umw[i] [0]
    verl_v_umw[i] = B_h2_umw[i] ['Hydrogen loss'] [v_umw[i]]                             #Verluste
    nutz_v_umw[i] = B_h2_umw [i]['Hydrogen use for heat gen'] [v_umw[i]]                 #Nutzung für Wärme (bei jetziger Technologieauswahl nicht zutreffend, eher untypisch, ggf. muss Rückumwnadlung berücksichtigt werden)

    #Ermittle skalierte GWP in Prozesskette von vorne nach hinten, Treten Verluste in einer Stufe auf, so wird das GWP der entsprecehnden Mehrproduktion der Vorstufen dieser Stufe zugeordnet
    #Beispiel: Treten Verluste im Transport auf, so wird das GWP welches in Erzeugung und Vorumwandlung für die benötigte Mehrproduktion entsteht der Transportstufe zugerechnet
    h2_prod_GWP_s[i] = h2_prod_GWP[i]
    h2_v_umw_GWP_s[i] = (1 + verl_v_umw[i]/100) * (1 + nutz_v_umw[i]/100) * h2_umw_GWP_ind[i].iloc[0] + ((1 + verl_v_umw[i]/100) * (1 + nutz_v_umw[i]/100) -1)* h2_prod_GWP_s[i]
    h2_transp_GWP_s[i] = (1 + verl_transp[i]/100) *(1+ nutz_transp[i]/100) * h2_transp_GWP_dir[i] + ((1 + verl_transp[i]/100) *(1+ nutz_transp[i]/100)-1) * (h2_v_umw_GWP_s[i] + h2_prod_GWP_s[i])
    h2_r_umw_GWP_s[i] = (1+ verl_r_umw[i]/100) * (1 + nutz_r_umw[i]/100) * h2_umw_GWP_ind[i].iloc[1] + ((1+ verl_r_umw[i]/100) * (1 + nutz_r_umw[i]/100)-1) * (h2_transp_GWP_s[i] + h2_v_umw_GWP_s[i] + h2_prod_GWP_s[i])
    #Annahme, dass verlorener transportierter Wasserstoff über gesamte Strecke transpoertiert wird. Korrekte Lösung über Integralrechnung möglich

# 5. Results and further analyses

Here, all results are calculated and prepared in tables and a plot. In the results box, you can see a plot of the GWP of the hydrogen production, the conversion, the transportation, and the reconversion of all routes. In the further analyses box, you can see more detailed results as the GWP of the electricity used in the production process or direct CO<sub>2</sub>-eq. emissions.

Below the graphs, there are more detailed information about the different considered stages and the emissions generated therein based on the routes you have selected. 

In [20]:
### Output

##Erstellen von Datenarrays für Plot und Tabelle

#GWP aller Stufen
h2_prod_GWP_s_data = np.array([h2_prod_GWP_s[key]['GWP'] for key in h2_prod_GWP_s])
h2_v_umw_GWP_s_data = np.array([h2_v_umw_GWP_s [key] ['GWP'] for key in h2_v_umw_GWP_s])
h2_transp_GWP_s_data = np.array([h2_transp_GWP_s [key] ['GWP'] for key in h2_transp_GWP_s])
h2_r_umw_GWP_s_data = np.array([h2_r_umw_GWP_s [key] ['GWP'] for key in h2_r_umw_GWP_s])

## Bei Produktion in Deutschland gibt es nur GWP in der H2 Produktion, da ansonsten keine Umwandlungen und Transportlängen anfallen

for i in range (anzahl):
    if prod_land[i] == 'Germany':
        h2_v_umw_GWP_s_data[i] = 0
        h2_transp_GWP_s_data [i] = 0
        h2_r_umw_GWP_s_data [i] = 0

#GWP ges
GWP_ges = {}
for i in range (anzahl):
    GWP_ges[i] = h2_prod_GWP_s[i] + h2_v_umw_GWP_s[i] + h2_transp_GWP_s[i] + h2_r_umw_GWP_s[i]
GWP_ges_data = np.array([GWP_ges[key] ['GWP'] for key in GWP_ges])

#H2 Production
h2_prod_data = np.array([h2_prod[key] for key in h2_prod])
#Vorumwandlung
v_umw_data = np.array([v_umw[key] for key in v_umw])
#Transport
h2_transp_data = np.array([h2_transp [key] for key in h2_transp])
#Rückumwandlung
r_umw_data = np.array([r_umw[key] for key in r_umw])
#Produktionsland
prod_land_data = np.array([prod_land[key] for key in prod_land])
entfernung_data = np.array([entfernung [key] *1000 for key in entfernung])
#GWP elctricity mix
strom_GWP_de_data = np.array([strom_GWP_de[key]*1000 for key in strom_GWP_de])
strom_GWP_prod_data = np.array([strom_GWP_prod[key]*1000 for key in strom_GWP_prod])
#Verluste
verl_tot = {}
for i in range (anzahl):
    verl_tot[i] = ((1+verl_v_umw[i]/100)*(1+verl_transp[i]/100)*(1+verl_r_umw[i]/100)-1)*1000
verl_tot_data = np.array([verl_tot[key] for key in verl_tot])
#Utilization for reconversion
nutz_r_umw_data = np.array([10* nutz_r_umw [key] for key in nutz_r_umw])
#Default Strommixe, zunächst Übernehmen der Values aus den Text Widgets, dann Umwandlen zu Array
def_strom_prod_i = {}
def_strom_de_i = {}
for i in range (anzahl):
    def_strom_prod_i[i] = def_strom_prod[i].value
    def_strom_de_i[i] = def_strom_de[i].value
def_strom_prod_data = np.array([def_strom_prod_i[key] for key in def_strom_prod_i])
def_strom_de_data = np.array([def_strom_de_i[key] for key in def_strom_de_i])
#Direct CO2 Emssionen in H2 Production
GWP_CO2_dir_h2_prod_data = [O_h2_prod[key]['CO2 emission'] for key in O_h2_prod]
#Strom für H2 Produktion, Wirkungsgrad ist vorher bereits berücksichtigt
el_h2_prod_data = [B_h2_prod[key]['Electricity (vor Wirkungsgrad)'] + B_h2_prod[key]['Electricity'] for key in B_h2_prod]
GWP_el_h2_prod_data = el_h2_prod_data * strom_GWP_prod_data/1000   

#Runden von Daten auf 2 Nachkommastellen
for i in range (anzahl):
    GWP_ges_data [i] = round(GWP_ges_data[i],2)
    h2_prod_GWP_s_data[i] = round(h2_prod_GWP_s_data[i],2)
    h2_v_umw_GWP_s_data[i] = round(h2_v_umw_GWP_s_data[i],2)
    h2_transp_GWP_s_data[i] = round(h2_transp_GWP_s_data[i],2)
    h2_r_umw_GWP_s_data[i] = round(h2_r_umw_GWP_s_data[i],2)
    nutz_r_umw_data[i] = round(nutz_r_umw_data[i],2)
    verl_tot_data[i] = round(verl_tot_data[i], 2)
    entfernung_data[i] = round(entfernung_data[i], 2)
    strom_GWP_prod_data[i] = round(strom_GWP_prod_data[i], 0)
    strom_GWP_de_data[i] = round(strom_GWP_de_data[i], 0)
    GWP_CO2_dir_h2_prod_data[i] = round(GWP_CO2_dir_h2_prod_data[i], 2)
    el_h2_prod_data[i] = round(el_h2_prod_data[i], 2)
    GWP_el_h2_prod_data[i] = round(GWP_el_h2_prod_data[i], 2)

#Beschreibung der Routen
anzahl_liste = {}
for i in range (anzahl):
    anzahl_liste[i] = 'Route ' + str(i+1)
x=np.array([anzahl_liste [key] for key in anzahl_liste])

# Erstellen der Tabellen

#Basic input data
basic_data = {}
basic_data = [prod_land_data, h2_prod_data, v_umw_data, h2_transp_data, r_umw_data, GWP_ges_data]

basic_tabelle = pd.DataFrame(basic_data)
basic_tabelle.index = ['Production country' , 'H2 production', 'H2 conversion', 'H2 transportation', 'H2 reconversion', 'GWP total [kg CO2 eq/ kg H2]' ] 
basic_tabelle.columns = [anzahl_liste [key] for key in anzahl_liste]


basic_tabelle_widget = widgets.Output()
with basic_tabelle_widget:
    display(basic_tabelle)


adv_data = [prod_land_data, GWP_ges_data,h2_prod_data, h2_prod_GWP_s_data, GWP_CO2_dir_h2_prod_data,el_h2_prod_data,GWP_el_h2_prod_data, 
v_umw_data, h2_v_umw_GWP_s_data, h2_transp_data, h2_transp_GWP_s_data, r_umw_data, h2_r_umw_GWP_s_data,
nutz_r_umw_data, verl_tot_data, entfernung_data, def_strom_prod_data, strom_GWP_prod_data, def_strom_de_data, strom_GWP_de_data,
]

adv_tabelle = pd.DataFrame(adv_data)

adv_tabelle.index = ['Production country' , 'GWP total [kg CO2 eq/ kg H2]', 'H2 production', 'GWP H2 production [kg CO2 eq/ kg H2]', 'Direct CO2 emissions H2 production [kg CO2/ kg H2]','Electricity used for H2 production [kWh/ kg H2]' , 'GWP el. used for H2 production [kg CO2 eq/ kg H2]',
'H2 conversion', 'GWP H2 conversion [kg CO2 eq/ kg H2]',
'H2 transportation', 'GWP H2 transportation [kg CO2 eq/ kg H2]', 'H2 reconversion', 'GWP H2 reconversion [kg CO2 eq/ kg H2]',
'H2 util. for H2 reconversion [g H2/ kg H2]', 'Total H2 losses [g H2/kg H2]', 'Travel distance [km]', 'El. mix prod. country (basic input)', 'GWP el. mix prod. country [g CO2 eq/ kWh]', 'El. mix Germany (basic input)', 'GWP el. mix Germany [g CO2 eq/ kWh]'
]

adv_tabelle.columns = [anzahl_liste [key] for key in anzahl_liste]

adv_tabelle_widget = widgets.Output()
with adv_tabelle_widget:
    display(adv_tabelle)

# Plot des Diagrams
diagramm_widget = widgets.Output()

with diagramm_widget:
    plt.bar(x, h2_prod_GWP_s_data, color = 'r')
    plt.bar(x, h2_v_umw_GWP_s_data, bottom = h2_prod_GWP_s_data, color ='b')
    plt.bar(x, h2_transp_GWP_s_data, bottom = h2_prod_GWP_s_data + h2_v_umw_GWP_s_data, color = 'y')
    plt.bar(x, h2_r_umw_GWP_s_data, bottom = h2_prod_GWP_s_data + h2_v_umw_GWP_s_data + h2_transp_GWP_s_data, color = 'g' )
    plt.legend(['H2 production ', 'H2 conversion', 'H2 transportation ', 'H2 reconversion'])
    plt.ylabel("kg CO2 eq / kg H2")
    #plt.title('GWP of selected hydrogen production and import routes')
    plt.show()

##Boxen und Layout

#Layout Funktion
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='20px 20px 20px 20px',
        padding='10px 10px 10px 10px'
     )

#VBoxen erstellen und Layout Funktion anwenden
diagram_widget_VBox = widgets.VBox([widgets.Label('GWP of selected hydrogen production and import routes:'), diagramm_widget])
diagram_widget_VBox.layout = make_box_layout()

basic_tabelle_widget_VBox = widgets.VBox([widgets.Label('Input data and total GWP:'), basic_tabelle_widget])
basic_tabelle_widget_VBox.layout = make_box_layout()

adv_tabelle_widget_VBox = widgets.VBox([widgets.Label('See all data and results in this table:'), adv_tabelle_widget])
adv_tabelle_widget_VBox.layout = make_box_layout()

ergebnisse = [widgets.VBox([widgets.Label('See your input data and a diagram of the GWP of the selected on this sheet. Go to the next sheet for further analyses.'),
    widgets.VBox([diagram_widget_VBox, basic_tabelle_widget_VBox])]),
    adv_tabelle_widget_VBox
    ]

ergebnisse_acc = widgets.Accordion()
ergebnisse_acc.children = ergebnisse
ergebnisse_acc.set_title(0, 'Results overview')
ergebnisse_acc.set_title(1, 'Detailed analyses')
display(ergebnisse_acc)

Accordion(children=(VBox(children=(Label(value='See your input data and a diagram of the GWP of the selected o…

### Hydrogen production

There is indirect GWP caused through the input flows and the material used in the processes and direct GWP through direct emissions of the processes. 

Especially for the **PEM electrolysis**, the indirect GWP is affected through the efficiency of the electrolysis and the electricity mix in the hydrogen production country. Moreover, the full load hours and the efficiency of the PEM electrolysis affect the overall hydrogen produced in the lifetime of the PEM stack and plant. An increase of the hydrogen produced in the lifetime decrease the GWP per kg hydrogen caused throgh the used material in the production as the GWP for the PEM stack and plant production are allocated on the hydrogen produced in their lifetime.

For the **steam methane reforming** and the **coal gasification**, direct emissions are most relevant as a lot of CO<sub>2</sub>-eq. is directly emitted in these processes. For instance, there is about 18.4 kg CO<sub>2</sub>-eq. emitted through the gasification of coal by steam and air to hydrogen. By using different carbon capturing and storage (CCS) methods, the most CO<sub>2</sub>-eq. can be captured. However, CCS can be expensive and energy consuming and therefore decreases the efficiency.

Besides the alternatives you can choose in this tool, there are also other ways to produce hydrogen like other electrolysis processes (e.g. alkaline electrolysis or solid oxide fuel cell) or pyrolysis and solar hydrogen generation which are not added yet.


### Hydrogen conversions

There is always a conversion step in the production country required to prepare hydrogen for transportation and a reconversion in Germany. Note, that your choosen tranportation method determines the conversion steps. For instance, a hydrogen compression and relaxation is performed if the hydrogen is transported by pipeline - when transporting hydrogen liquefied a liquefaction and a regasification are conducted. The liquefaction of hydrogen is very energy intensive as the hydrogen has to be cooled down to 20 K (-253 °C). In the liquefaction proccess used in this model, electricity of about 20 % of the hydrogen heating value is consumed.

There are also losses of hydrogen occuring in the conversion processes. Moreover, the energy of hydrogen can be utilized in endothermic conversion processes (especially in the LOHC dehydrogenation when the hydrogen is desorbed from the liquefied organic hydrogen carrier (LOHC) and a temperature of about 300 °C is required). 




### Hydrogen transportation

The GWP emission for hydrogen transportation depends on the choosen  method and on the travel distance. The liquefied transportation by ship has the lowest GWP. However, the GWP of the conversions are the highest due to the liqufaction. Therefore, there is usually an intersection at a specific distance, when the liquefied transportation becomes the most environment friendly - in terms of GWP - method. For shorter distances, pipelines usualy have the lowest GWP when considering the GWP of the conversion steps. The transportation through LOHC is only in a few cases environment friendly. A road transport only make scene for short distances when no pipeline is available. There are also losses of hydrogen occuring during transportation. A hydrogen utilization for transportation is not implemented in this model so far.


### Hydrogen losses and utilization 

Depending on the loss and the utilization on the selected routes, the additional amount of hydrogen has to be produced to utilize one kg of hydrogen at the destination in German. Here, the GWP caused through the additional hydrogen production is not added to the GWP of the hydrogen production but added to the GWP of the process where the loss or the utilization occurs. For instance, if 16 grams of hydrogen per kg hydrogen are getting lost in the liquefaction process, the GWP of 16 grams hydrogen production are added to the the GWP of the hydrogen conversion. 

The later the losses or the utilization occur, the higher the complexity gets: in this model, 300 grams of hydrogen per kg hydrogen are utilzed for the LOHC dehydrogenation (reconversion process). Therefore 300 grams more have to be transported. Now, there also occure additional losses in transportation wich also have to be allocated to the LOHC dehydrogenation and so on. In the end, more than additional 300 grams of hydrogen have to be produced due to the utilization in the LOHC dehydrogenation process - this additional GWP is added to the GWP of the LOHC dehydrogenation process.